In [ ]:
#!pip install pandas

In [810]:
import pandas as pd

In [719]:
sch_data = 'SCH-ZZ-20231208_035117.xlsx'      #Importing schedule data
inv_data = 'INV-ZZ-20231208_041852.xlsx'      #Importing inventory data

In [720]:
sch = pd.read_excel(sch_data)
inv = pd.read_excel(inv_data)

In [811]:
df1 = pd.DataFrame(sch)         #Converting excel data into dataframe
df2 = pd.DataFrame(inv)         

In [28]:
df2['Bool'] = 1   #Creating a new column in df2 and naming it as Bool and assigning all values as 1
                  #Bool value will be 0 for the flights which are cancelled

In [29]:
import numpy as np
cancelled_flights = [0]       #If the first flight is cancelled

In [30]:
for i in cancelled_flights:
    df2.at[i, 'Bool'] = 0     #Assigning a bool value of 0 to the cancelled flights
print(cancelled_flights)

[0]


In [31]:
threshold1 = pd.Timedelta(hours=6)
threshold2 = pd.Timedelta(hours=12)
threshold3 = pd.Timedelta(hours=24)
threshold4 = pd.Timedelta(hours=48)
threshold5 = pd.Timedelta(hours=72)
zero_time = pd.Timedelta(hours=0)
inf_time = pd.Timedelta(weeks=52)

In [32]:
#Algorithm for flight scoring based on arrival, departure time, arrival, departure time etc. 
for j in range(len(cancelled_flights)):
    new_col = f'Flight_Score_{cancelled_flights[j]}'
    new_col2 = f'Quality Score Grade_{cancelled_flights[j]}'
    df2[new_col] = 0                 #New column in the dataframe for scoring cancelled flights
    df2[new_col2] = 'NaN'            #Each flights having a bool value of 1 will have a score 
                                     #corresponding to each cancelled flight
    
    for i in range(len(df2)):
        if df2.loc[i, 'Bool'] == 1:  #Only scoring non-cancelled flights
            if df2.loc[i, 'DepartureAirport'] == df2.loc[cancelled_flights[j], 
            'DepartureAirport'] and df2.loc[i, 'ArrivalAirport'] == df2.loc[cancelled_flights[j], 'ArrivalAirport']:
               df2.loc[i, new_col] = 40 
               diff_dep = df2.loc[i, 'DepartureDateTime'] - df2.loc[cancelled_flights[j], 'DepartureDateTime']
               diff_arr = df2.loc[i, 'ArrivalDateTime'] - df2.loc[cancelled_flights[j], 'ArrivalDateTime']
               if((diff_dep <= threshold4) and (diff_dep >= zero_time)):                            
                    df2.loc[i, new_col] = df2.loc[i, new_col] + 30                                  
                    if((diff_dep <= threshold3) and (diff_dep >= zero_time)):                       
                        df2.loc[i, new_col] = df2.loc[i, new_col] + 10   #Adding only 10 because there is a nested-if 
                        if((diff_dep <= threshold2) and (diff_dep >= zero_time)):
                            df2.loc[i, new_col] = df2.loc[i, new_col] + 10
                            if((diff_dep <= threshold1) and (diff_dep >= zero_time)):
                                df2.loc[i, new_col] = df2.loc[i, new_col] + 20
               if((diff_arr <= threshold4) and (diff_arr >= zero_time)):                            
                    df2.loc[i, new_col] = df2.loc[i, new_col] + 30                                    
                    if((diff_arr <= threshold3) and (diff_arr >= zero_time)):                        
                        df2.loc[i, new_col] = df2.loc[i, new_col] + 10
                        if((diff_arr <= threshold2) and (diff_arr >= zero_time)):
                            df2.loc[i, new_col] = df2.loc[i, new_col] + 10
                            if((diff_arr <= threshold1) and (diff_arr >= zero_time)):
                                df2.loc[i, new_col] = df2.loc[i, new_col] + 20
               for k in range(len(df1)):
                    if (df2.loc[i, 'ScheduleId'] == df1.loc[k, 'ScheduleID']):
                        tail_number = df1.loc[k, 'AircraftTailNumber']
               for k in range(len(df1)):
                    if (df2.loc[cancelled_flights[j], 'ScheduleId'] == df1.loc[k, 'ScheduleID']):
                        if (df1.loc[k, 'AircraftTailNumber'] == tail_number):
                            df2.loc[i, new_col] = df2.loc[i, new_col] + 50

            #Quality grading of flights acc. to ruleset provided
            if(df2.loc[i, new_col] < 150 and df2.loc[i, new_col] > 0):
                df2.loc[i, new_col2] = 'D'
            if(df2.loc[i, new_col] >= 150):
                df2.loc[i, new_col2] = 'C'
            if(df2.loc[i, new_col] >= 180):
                df2.loc[i, new_col2] = 'B'
            if(df2.loc[i, new_col] >= 200):
                df2.loc[i, new_col2] = 'A'

df2 = pd.concat([df2], axis=1)

In [542]:
filt1 = (df2[f'Quality Score Grade_{cancelled_flights[0]}'] == 'A')
filt2 = (df2[f'Quality Score Grade_{cancelled_flights[0]}'] == 'B')
filt3 = (df2[f'Quality Score Grade_{cancelled_flights[0]}'] == 'C')
filt4 = (df2['Oversold'] < 0)       #Considering only those flights which have vacant seats
Available_flights = df2.loc[(filt1 | filt2 | filt3) & filt4]

In [543]:
Available_flights

,InventoryId,ScheduleId,CarrierCode,Dep_Key,FlightNumber,AircraftType,DepartureDate,DepartureDateTime,ArrivalDateTime,DepartureAirport,...,EC_BookedInventory,EC_Oversold,EC_AvailableInventory,FC_CD,BC_CD,PC_CD,EC_CD,Bool,Flight_Score_0,Quality Score Grade_0
1,INV-ZZ-8029879,SCH-ZZ-3400127,ZZ,ZZ20240506AMDHYD2223.0,2223,Boeing 777,2024-05-06,2024-05-06 10:52:00,2024-05-06 19:30:00,AMD,...,106,64,96,"{'F': 25, 'P': 17}","{'C': 34, 'J': 26, 'Z': 26}","{'Q': 38, 'R': 25, 'S': 13, 'T': 13, 'H': 25, ...","{'Y': 13, 'A': 13, 'B': 6, 'D': 6, 'E': 6, 'G'...",1,170,C
2,INV-ZZ-6852033,SCH-ZZ-3400127,ZZ,ZZ20240507AMDHYD2223.0,2223,Boeing 777,2024-05-07,2024-05-07 10:52:00,2024-05-07 19:30:00,AMD,...,114,56,83,"{'F': 25, 'P': 17}","{'C': 34, 'J': 26, 'Z': 26}","{'Q': 38, 'R': 25, 'S': 13, 'T': 13, 'H': 25, ...","{'Y': 13, 'A': 13, 'B': 6, 'D': 6, 'E': 6, 'G'...",1,150,C


In [520]:
F = []          #Array for storing score for each alternate flight
Av_Fl = []      #Array for storing index of each alternate flight in the inventory dataframe
for i in range(len(Available_flights)):
    ix = Available_flights.index[i]
    Av_Fl.append(ix)
    F.append(df2.loc[ix, f'Quality Score Grade_{cancelled_flights[0]}'])     
    #Change the index based on the index of the flight cancelled

#Converting Quality Score into numerical scores
for i in range(len(F)):
    if F[i] == 'A':
        F[i] = 4     
    if F[i] == 'B':
        F[i] = 3
    if F[i] == 'C':
        F[i] = 2
    if F[i] == 'D': 
        F[i] = 1
        
V_S = []       #Array for storing number of available seats in each alternating flight
for i in range(len(Available_flights)):
    ix = Available_flights.index[i]
    V_S.append(-df2.loc[ix, 'Oversold'])

In [528]:
#Scaling down the values in V_S 
for i in range(len(V_S)):
    V_S[i] = V_S[i]/10
    V_S[i] = np.floor(V_S[i])   #Floor value so that no. of passengers accommodated
print(V_S)                      #in each flight is less than the flight's capacity

[15.0, 14.0]


In [ ]:
#We have:
#F containing the scores for each alternate flight
#Av_Fl containing the index of each alternate flight in the inventory dataframe
#V_S containing the number of available seats in each alternating flight
#All the above 3 arrays have the same length. The corresponding indices represent the same flights

In [320]:
PNR = 'PNRB-ZZ-20231208_062017.xlsx'     #Importing the PNR Data
Pass = 'PNRP-ZZ-20231208_111136.xlsx'    #Importing the Passenger Data

In [321]:
PNR1 = pd.read_excel(PNR)          
Pass1 = pd.read_excel(Pass)

In [322]:
PNR_DF = pd.DataFrame(PNR1)             #Converting excel data into dataframe
Pass_DF = pd.DataFrame(Pass1)

In [323]:
Pass_DF['SSR_CODE_CD1'] = Pass_DF['SSR_CODE_CD1'].fillna(0)
Pass_DF['TierLevel'] = Pass_DF['TierLevel'].fillna(0)

In [324]:
#Function that returns the PNRs that are affected based on flight no., and departure date of the cancelled flight
def Affected_PNR(FLT_NUM, DEP_DATE, PNR_DF):
    AffectedPNR = PNR_DF.loc[(PNR_DF['FLT_NUM'] == FLT_NUM) & (PNR_DF['DEP_DT']==DEP_DATE)]
    return AffectedPNR

In [693]:
#Dataframe consisting of affected PNRs
aff = Affected_PNR(df2.loc[cancelled_flights[0],'FlightNumber'], df2.loc[cancelled_flights[0],'DepartureDate'], PNR_DF)

In [326]:
#Scoring Passengers

In [327]:
departure_list = PNR_DF['ORIG_CD'].to_list()
arrival_list = PNR_DF['DEST_CD'].to_list()

city_pairs = [tuple([departure_list[i], arrival_list[i]]) for i in range(len(departure_list))]

fltnum_list = PNR_DF["FLT_NUM"].to_list()

citypair_dict = {fltnum_list[i] : city_pairs[i] for i in range(len(city_pairs))}

In [329]:
paxcnt_wt = 50
connect_wt = 100
grp_threshold = 7
grp_wt = 500
class_wt = {
    'FirstClass' : 2000, 'BusinessClass' : 1750, 
    'PremiumEconomyClass' : 1500, 'EconomyClass' : 1250
}
# The airlines can vary the weights as per their rules set and have the choice to disregard certain factors as per their choice.

In [330]:
def get_recloc_score(df):
    recloc_sclist = []
    recloc_score = 0
    for i in range(len(df['RECLOC'])):
        recloc_score += df['PAX_CNT'][i]*paxcnt_wt + class_wt[df['COS_CD'][i]]
        if df['PAX_CNT'][i] >= grp_threshold:
            recloc_score += grp_wt
        recloc_sclist.append(recloc_score)
        recloc_score = 0

    df['RECLOC_SCORE'] = recloc_sclist

    return df    

#Due to multiple passengers having the same recloc, We have split calculating passenger 
#score into two parts : recloc_Score and passenger_score. The recloc score will initialise
#the pax_score variable and build up from there.

In [331]:
get_recloc_score(PNR_DF)

,RECLOC,CREATION_DTZ,DEP_KEY,ACTION_CD,COS_CD,SEG_SEQ,SEG_TOTAL,PAX_CNT,CARRIER_CD,FLT_NUM,ORIG_CD,DEST_CD,DEP_DT,DEP_DTML,ARR_DTML,DEP_DTMZ,ARR_DTMZ,RECLOC_SCORE
0,DRGS80,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,4,3,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2150
1,YEZQ47,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,2,4,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2200
2,JDDM40,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,3,2,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2100
3,AVFF16,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,4,1,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2050
4,DBLE79,2024-03-02 16:35:00,ZZ20240403BLRCCU2504,GN,FirstClass,1,2,2,ZZ,2504,BLR,CCU,2024-04-03,2024-04-03 15:35:00,2024-04-03 18:17:00,2024-04-03 10:05:00,2024-04-03 12:47:00,2100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78179,CJEL38,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,4,7,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,2100
78180,NDXM23,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,1,9,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,2200
78181,RIUQ17,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,4,5,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,1500
78182,QLFS50,2023-08-20 13:28:00,ZZ20240630AMDBLR4957,HL,EconomyClass,1,2,1,ZZ,4957,AMD,BLR,2024-06-30,2024-06-30 10:19:00,2024-06-30 21:50:00,2024-06-30 04:49:00,2024-06-30 16:20:00,1300


In [332]:
recloc_dict = {PNR_DF['RECLOC'][i] : PNR_DF['RECLOC_SCORE'][i] for i in range(len(PNR_DF['RECLOC']))}

In [333]:
loyalty_dict = {'Platinum' : 1800, 'Gold' : 1600, 'Silver' : 1500, 0 : 0}
ssr_wt = 200
# The airlines can vary the weights as per their rules set and have the choice to disregard certain factors as per their choice.

In [334]:
def get_pnr_score(df):
    pnr_sclist = []
    pnr_score = 0
    for i in range(len(df['RECLOC'])):
        pnr_score += recloc_dict[df['RECLOC'][i]] + loyalty_dict[df['TierLevel'][i]]
        if df['SSR_CODE_CD1'][i] != 0:
            pnr_score += ssr_wt
        
        pnr_sclist.append(pnr_score)
        pnr_score = 0

    df['PNR_SCORE'] = pnr_sclist

    return df    
    

In [335]:
get_pnr_score(Pass_DF)

,RECLOC,CREATION_DTZ,CUSTOMER_ID,LAST_NAME,FIRST_NAME,NATIONALITY,DOB,CONTACT_PH_NUM,CONTACT_EMAIL,DOC_TYPE,DOC_ID,SPECIAL_NAME_CD2,SSR_CODE_CD1,SPECIAL_NAME_CD1,FF_NUM,TierLevel,PNR_SCORE
0,DRGS80,2024-03-02 16:35:00,VNQ9748,Mitchell,Christina,CA,2003-06-01,1-769-506-8397,christinamitchell1@example.com,Driving License,299196000000,NaN,0,ADT,4.276771e+09,Gold,3750
1,DRGS80,2024-03-02 16:35:00,BAP1926,Parker,Rebecca,CA,2015-08-18,1-947-699-5143,rebeccaparker2@example.com,Passport,11901984430,NaN,0,CHD,8.493470e+09,Platinum,3950
2,DRGS80,2024-03-02 16:35:00,ALV9938,Swamy,Vardaniya,IN,1945-02-23,6102977873,vardaniyaswamy3@example.com,Govt-Id,4283733463,NaN,0,S65,6.731611e+09,Platinum,3950
3,YEZQ47,2024-03-02 16:35:00,CEU7075,Edwin,Stuvan,IN,2003-06-18,5791967032,stuvanedwin1@example.com,Passport,139267560,NaN,0,ADT,7.214819e+09,Platinum,4000
4,YEZQ47,2024-03-02 16:35:00,WFG9647,Hernandez,Andrea,CA,1969-07-03,+1 (378) 291-0325,andreahernandez2@example.com,Govt-Id,5459762750,NaN,0,ADT,NaN,0,2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152508,RIUQ17,2023-08-20 13:28:00,NQF8798,Shankar,Divit,IN,1981-08-22,(347) 703-8971,divitshankar3@example.com,Driving License,169860000000,NaN,0,ADT,NaN,0,1500
152509,RIUQ17,2023-08-20 13:28:00,RJI4602,Henderson,Autumn,US,2013-09-14,(371) 218-4438,autumnhenderson4@example.com,Govt-Id,7100653216,NaN,0,CHD,5.882689e+09,Gold,3100
152510,RIUQ17,2023-08-20 13:28:00,VPC0920,Ramanathan,Hridaan,IN,1944-09-06,(437) 839-4309,hridaanramanathan5@example.com,Driving License,715385534,NaN,0,S65,NaN,0,1500
152511,QLFS50,2023-08-20 13:28:00,BIB2224,Baria,Taimur,IN,2010-06-19,(771) 284-3525,taimurbaria1@example.com,Govt-Id,4431751697,NaN,0,CHD,3.701390e+09,Gold,2900


In [377]:
aff = Affected_PNR(df2.loc[cancelled_flights[0],'FlightNumber'], df2.loc[cancelled_flights[0],'DepartureDate'], PNR_DF)

In [378]:
#Dataframe consisting of affected passengers
aff_pax = pd.merge(Pass_DF, aff[['RECLOC']], on='RECLOC', how='inner')

In [379]:
#Grouping passengers based on PNR_Score
for i in range(len(aff_pax)):
    score = aff_pax.loc[i, 'PNR_SCORE']
    
    if score >= 3500:
        aff_pax.loc[i, 'PNR_SCORE'] = 5
    elif score >= 3000:
        aff_pax.loc[i, 'PNR_SCORE'] = 4
    elif score >= 2500:
        aff_pax.loc[i, 'PNR_SCORE'] = 3
    elif score >= 2000:
        aff_pax.loc[i, 'PNR_SCORE'] = 2
    else:
        aff_pax.loc[i, 'PNR_SCORE'] = 1


In [383]:
value_counts = aff_pax['PNR_SCORE'].value_counts()

In [384]:
# Convert the Series to a DataFrame
freq_df = pd.DataFrame({'PNR_SCORE': value_counts.index, 'Frequency': value_counts.values})

In [385]:
freq_df

,PNR_SCORE,Frequency
0,5,90
1,4,64
2,1,50
3,2,48
4,3,3


In [533]:
Score = []         #Array for storing PNR scores
G = []             #Array for storing number of passengers 
                   #corresponding to the PNR Score
for i in range(len(freq_df)):
    Score.append(freq_df.loc[i, 'PNR_SCORE'])
    G.append(freq_df.loc[i, 'Frequency'])
print(Score)
print(G)

[5, 4, 1, 2, 3]
[90, 64, 50, 48, 3]


In [534]:
#Scaling down the values in G
for i in range(len(G)):
    G[i] = G[i]/10
    G[i] = np.ceil(G[i])   #Ceil value so that number of passengers accommodated in
                           #a flight is always less than than the flight's capacity
print(G)

[9.0, 7.0, 5.0, 5.0, 1.0]


In [389]:
#Importing class from PNR Data to Passenger Data
aff_pax['Class'] = 'NaN'

for i in range(len(aff)):
    recloc_aff = aff.loc[aff.index[i], 'RECLOC']
    class_aff = aff.loc[aff.index[i], 'COS_CD']

    aff_pax.loc[aff_pax['RECLOC'] == recloc_aff, 'Class'] = class_aff

In [ ]:
#Solving our QUBO problem using Quantum Solvers

In [ ]:
#!pip install qiskit
#!pip install qiskit_algorithms
#!pip install qiskit_optimization

In [404]:
from qiskit_algorithms import QAOA, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import COBYLA, SPSA
from qiskit.primitives import Sampler
from qiskit_optimization.algorithms import (
    MinimumEigenOptimizer,
    SolutionSample,
    OptimizationResultStatus,
)
from qiskit_optimization import QuadraticProgram
from qiskit.visualization import plot_histogram
from typing import List, Tuple
import numpy as np
from qiskit_optimization.translators import from_docplex_mp
from qiskit_optimization.converters import LinearInequalityToPenalty, QuadraticProgramToQubo

In [536]:
#Formulating the problem
qp = QuadraticProgram()

#Defining variables
for j in range(len(F)):
    for i in range(len(Score)):
        qp.binary_var(name=f'x_{i}_{j}')

#Objective function
linear_dict = {f'x_{i}_{j}': F[j] * Score[i] for i in range(len(Score)) for j in range(len(F))}
qp.maximize(linear=linear_dict)

#Add constraints
#Passenger Constraint (Each passenger(group here) is assigned one flight)
for k in range(len(Score)):
    constraint1 = {f'x_{k}_{j}': 1 for j in range(len(F))}
    qp.linear_constraint(linear=constraint1, sense='<=', rhs=1)

#Flight Constraint (Flights shouldn't accommodate more than it can carry/fly)
for k in range(len(F)):
    constraint2 = {f'x_{i}_{k}': G[i] for i in range(len(Score))}
    qp.linear_constraint(linear=constraint2, sense='<=', rhs=V_S[k])

print(qp.prettyprint())

Problem name: 

Maximize
  10*x_0_0 + 10*x_0_1 + 8*x_1_0 + 8*x_1_1 + 2*x_2_0 + 2*x_2_1 + 4*x_3_0
  + 4*x_3_1 + 6*x_4_0 + 6*x_4_1

Subject to
  Linear constraints (7)
    x_0_0 + x_0_1 <= 1  'c0'
    x_1_0 + x_1_1 <= 1  'c1'
    x_2_0 + x_2_1 <= 1  'c2'
    x_3_0 + x_3_1 <= 1  'c3'
    x_4_0 + x_4_1 <= 1  'c4'
    9*x_0_0 + 7*x_1_0 + 5*x_2_0 + 5*x_3_0 + x_4_0 <= 15  'c5'
    9*x_0_1 + 7*x_1_1 + 5*x_2_1 + 5*x_3_1 + x_4_1 <= 14  'c6'

  Binary variables (10)
    x_0_0 x_1_0 x_2_0 x_3_0 x_4_0 x_0_1 x_1_1 x_2_1 x_3_1 x_4_1



In [537]:
#Converting our problem to QUBO using optimization converters
ineq2pen_2 = QuadraticProgramToQubo()
qubo = ineq2pen_2.convert(qp)
print(qubo.prettyprint())
#All the constraints have been converted to penalty terms and added 
#to the objective function as well as slack variables are introduced

Problem name: 

Minimize
  366*c5@int_slack@0^2 + 1464*c5@int_slack@0*c5@int_slack@1
  + 2928*c5@int_slack@0*c5@int_slack@2 + 5856*c5@int_slack@0*c5@int_slack@3
  + 1464*c5@int_slack@1^2 + 5856*c5@int_slack@1*c5@int_slack@2
  + 11712*c5@int_slack@1*c5@int_slack@3 + 5856*c5@int_slack@2^2
  + 23424*c5@int_slack@2*c5@int_slack@3 + 23424*c5@int_slack@3^2
  + 366*c6@int_slack@0^2 + 1464*c6@int_slack@0*c6@int_slack@1
  + 2928*c6@int_slack@0*c6@int_slack@2 + 5124*c6@int_slack@0*c6@int_slack@3
  + 1464*c6@int_slack@1^2 + 5856*c6@int_slack@1*c6@int_slack@2
  + 10248*c6@int_slack@1*c6@int_slack@3 + 5856*c6@int_slack@2^2
  + 20496*c6@int_slack@2*c6@int_slack@3 + 17934*c6@int_slack@3^2
  + 6588*x_0_0*c5@int_slack@0 + 13176*x_0_0*c5@int_slack@1
  + 26352*x_0_0*c5@int_slack@2 + 52704*x_0_0*c5@int_slack@3 + 29646*x_0_0^2
  + 61*x_0_0*x_0_1 + 46116*x_0_0*x_1_0 + 32940*x_0_0*x_2_0 + 32940*x_0_0*x_3_0
  + 6588*x_0_0*x_4_0 + 6588*x_0_1*c6@int_slack@0 + 13176*x_0_1*c6@int_slack@1
  + 26352*x_0_1*c6@int_sl

In [538]:
#Converting our QUBO problem to Ising Model 
H, offset = qubo.to_ising()
print("offset: {}".format(offset))
print("H:")
print(H)    #The Hamiltonian for our problem

offset: 75991.25
H:
SparsePauliOp(['IIIIIIIIIIIIIIIIIZ', 'IIIIIIIIIIIIZIIIII', 'IIIIIIIIIIIIIIIIZI', 'IIIIIIIIIIIZIIIIII', 'IIIIIIIIIIIIIIIZII', 'IIIIIIIIIIZIIIIIII', 'IIIIIIIIIIIIIIZIII', 'IIIIIIIIIZIIIIIIII', 'IIIIIIIIIIIIIZIIII', 'IIIIIIIIZIIIIIIIII', 'IIIIIIIZIIIIIIIIII', 'IIIIIIZIIIIIIIIIII', 'IIIIIZIIIIIIIIIIII', 'IIIIZIIIIIIIIIIIII', 'IIIZIIIIIIIIIIIIII', 'IIZIIIIIIIIIIIIIII', 'IZIIIIIIIIIIIIIIII', 'ZIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIZIZ', 'IIIIIIIIIIIIIIZIIZ', 'IIIIIIIIIIIIIZIIIZ', 'IIIIIIIIIIIIZIIIIZ', 'IIIIIIIZIIIIIIIIIZ', 'IIIIIIZIIIIIIIIIIZ', 'IIIIIZIIIIIIIIIIIZ', 'IIIIZIIIIIIIIIIIIZ', 'IIIIIIIIIIIIIIIZZI', 'IIIIIIIIIIIIIIZIZI', 'IIIIIIIIIIIIIZIIZI', 'IIIIIIIIIIIZIIIIZI', 'IIIIIIIZIIIIIIIIZI', 'IIIIIIZIIIIIIIIIZI', 'IIIIIZIIIIIIIIIIZI', 'IIIIZIIIIIIIIIIIZI', 'IIIIIIIIIIIIIIZZII', 'IIIIIIIIIIIIIZIZII', 'IIIIIIIIIIZIIIIZII', 'IIIIIIIZIIIIIIIZII', 'IIIIIIZIIIIIIIIZII', 'IIIIIZIIIIIIIIIZII', 'IIIIZIIIIIIIIIIZII', 'IIIIIIIIIIIIIZZIII', 'IIIIIIIIIZIIIIZIII

In [539]:
qaoa_mes = QAOA(sampler=Sampler(), optimizer=COBYLA(),reps=1)

In [540]:
qaoa = MinimumEigenOptimizer(qaoa_mes)  # using QAOA

In [541]:
qubo_result = qaoa.solve(qubo)
print(qubo_result.prettyprint())

objective function value: -30.0
variable values: x_0_0=0.0, x_1_0=1.0, x_2_0=0.0, x_3_0=1.0, x_4_0=1.0, x_0_1=1.0, x_1_1=0.0, x_2_1=1.0, x_3_1=0.0, x_4_1=0.0, c5@int_slack@0=0.0, c5@int_slack@1=1.0, c5@int_slack@2=0.0, c5@int_slack@3=0.0, c6@int_slack@0=0.0, c6@int_slack@1=0.0, c6@int_slack@2=0.0, c6@int_slack@3=0.0
status: SUCCESS


In [802]:
print("variable order:", [var.name for var in qubo_result.variables])
X = []
for i in range(min(4, len(qubo_result.samples))):
    sample = qubo_result.samples[i]

    # Extract 'x' array from the sample
    x_array = sample.x
    X.append(x_array[:len(Score) * len(F)])   #Omitting the slack variables

variable order: ['x_0_0', 'x_1_0', 'x_2_0', 'x_3_0', 'x_4_0', 'x_0_1', 'x_1_1', 'x_2_1', 'x_3_1', 'x_4_1', 'c5@int_slack@0', 'c5@int_slack@1', 'c5@int_slack@2', 'c5@int_slack@3', 'c6@int_slack@0', 'c6@int_slack@1', 'c6@int_slack@2', 'c6@int_slack@3']


In [803]:
X #List that contains the first 4 optimal solutions

[array([0., 1., 0., 1., 1., 1., 0., 1., 0., 0.]),
 array([0., 1., 1., 0., 1., 1., 0., 0., 1., 0.]),
 array([0., 1., 0., 0., 1., 1., 0., 0., 1., 0.]),
 array([1., 0., 0., 0., 0., 0., 1., 0., 1., 1.]),
 array([0., 1., 0., 1., 0., 1., 0., 0., 0., 1.])]

In [812]:
#Anslyzing first solution
n=1
X[n-1]

array([0., 1., 0., 1., 1., 1., 0., 1., 0., 0.])

In [813]:
aff_pax['Alt_Fli'] = 'NaN'      #New column for storing the assigned alternate flight to passengers

In [814]:
for i in range(len(X[n-1])):
    if X[n-1][i] == 1:
        alt_flight_number = (i // len(Score)) + 1
        group_index = (i % len(Score)) + 1
        assignment_info = f"Group {group_index} is assigned to Flight {alt_flight_number}" 
        print(assignment_info)
        condition = aff_pax['PNR_SCORE'] == Score[group_index - 1]
        aff_pax.loc[condition, 'Alt_Fli'] = Av_Fl[alt_flight_number - 1]       

Group 2 is assigned to Flight 1
Group 4 is assigned to Flight 1
Group 5 is assigned to Flight 1
Group 1 is assigned to Flight 2
Group 3 is assigned to Flight 2


In [815]:
aff_pax['Alt_Fli'].value_counts()  #No. of passengers in each flight

Alt_Fli
2    140
1    115
Name: count, dtype: int64

In [816]:
filt = (aff_pax['Alt_Fli'] != 'NaN')
Reaccommodated_List_df = aff_pax.loc[filt] #Dataframe that contains the passengers who were rescheduled

In [817]:
Reaccommodated_List_df.head()

,RECLOC,CREATION_DTZ,CUSTOMER_ID,LAST_NAME,FIRST_NAME,NATIONALITY,DOB,CONTACT_PH_NUM,CONTACT_EMAIL,DOC_TYPE,DOC_ID,SPECIAL_NAME_CD2,SSR_CODE_CD1,SPECIAL_NAME_CD1,FF_NUM,TierLevel,PNR_SCORE,Class,Alt_Fli,Assigned Class
0,YFIU24,2023-07-12 18:54:00,AAJ5569,Simpson,Christine,CA,1983-05-10,(931) 805-6794,christinesimpson1@example.com,Passport,54800342,NaN,0,ADT,NaN,0,1,EconomyClass,2,
1,YFIU24,2023-07-12 18:54:00,RWT4783,Brown,Michael,US,1937-10-01,(542) 239-1961,michaelbrown2@example.com,Passport,633725606,NaN,0,S65,9.221551e+09,Gold,4,EconomyClass,1,
2,YFIU24,2023-07-12 18:54:00,PKA3536,Brown,Eric,US,1938-11-05,(310) 411-9646,ericbrown3@example.com,Govt-Id,1709385440,NaN,0,S65,NaN,0,1,EconomyClass,2,
3,GDMM37,2023-04-15 09:10:00,MXI3729,Huynh,Cameron,US,1984-01-04,(994) 412-9858,cameronhuynh1@example.com,Govt-Id,4705551688,NaN,0,ADT,7.503245e+09,Silver,5,PremiumEconomyClass,2,
4,GDMM37,2023-04-15 09:10:00,QFV7801,Compton,Steve,CA,1934-11-25,(218) 532-5335,stevecompton2@example.com,Passport,342761628,NaN,0,S65,NaN,0,2,PremiumEconomyClass,1,


In [818]:
filt = (aff_pax['Alt_Fli'] == 'NaN')
Exception_List_df = aff_pax.loc[filt] #Dataframe that contains the passengers who couldn't be rescheduled

In [819]:
Exception_List_df.head()

,RECLOC,CREATION_DTZ,CUSTOMER_ID,LAST_NAME,FIRST_NAME,NATIONALITY,DOB,CONTACT_PH_NUM,CONTACT_EMAIL,DOC_TYPE,DOC_ID,SPECIAL_NAME_CD2,SSR_CODE_CD1,SPECIAL_NAME_CD1,FF_NUM,TierLevel,PNR_SCORE,Class,Alt_Fli,Assigned Class


In [842]:
print(f'{len(Reaccommodated_List_df)} passengers out of {len(aff_pax)} were reaccommodated')

255 passengers out of 255 were reaccommodated


In [820]:
aff_pax['Assigned Class'] = ''     #New column for storing the newly assigned class in alternate flights

In [821]:
#Alternate flight 1
f = 1
filt = (aff_pax['Alt_Fli'] == Av_Fl[f-1])
filtered = aff_pax.loc[filt]

In [822]:
filt = (aff_pax['Alt_Fli'] == Av_Fl[f-1])
filtered_pass_data_1 = aff_pax.loc[filt]

In [ ]:
# Assign classes based on 'Score' and class limits
def assign_class(row):
     global class_count
     for category, limit in class_limits.items():
           if class_count[category] < limit:
               class_count[category] += 1
               return category
     return None

In [823]:
filt_sorted_1 = filtered_pass_data_1.copy()  # Making a copy to avoid modifying the original DataFrame

# Sort the DataFrame based on 'Score' in descending order
filt_sorted_1 = filt_sorted_1.sort_values(by='PNR_SCORE', ascending=False)

# Define class categories and corresponding limits
class_categories = ['FirstClass', 'BusinessClass', 'PremiumEconomyClass', 'EconomyClass']
class_limits = {'FirstClass': df2.loc[Av_Fl[f-1], 'FC_Oversold'], 'BusinessClass': df2.loc[Av_Fl[f-1], 'BC_Oversold'], 'PremiumEconomyClass': df2.loc[Av_Fl[f-1], 'PC_Oversold'], 'EconomyClass': df2.loc[Av_Fl[f-1], 'EC_Oversold']} 

# Initialize count for each class
class_count = {category: 0 for category in class_categories}

# Create a new column 'Assigned Class'
filt_sorted_1['Assigned Class'] = filt_sorted_1.apply(assign_class, axis=1)

In [824]:
#Alternate flight 2
f = 2
filt = (aff_pax['Alt_Fli'] == Av_Fl[f-1])
filtered = aff_pax.loc[filt]

In [825]:
filt = (aff_pax['Alt_Fli'] == Av_Fl[f-1])
filtered_pass_data_2 = aff_pax.loc[filt]

In [826]:
filt_sorted_2 = filtered_pass_data_2.copy()  # Making a copy to avoid modifying the original DataFrame

# Sort the DataFrame based on 'Score' in descending order
filt_sorted_2 = filt_sorted_2.sort_values(by='PNR_SCORE', ascending=False)

# Define class categories and corresponding limits
class_categories = ['FirstClass', 'BusinessClass', 'PremiumEconomyClass', 'EconomyClass']
class_limits = {'FirstClass': df2.loc[Av_Fl[f-1], 'FC_Oversold'], 'BusinessClass': df2.loc[Av_Fl[f-1], 'BC_Oversold'], 'PremiumEconomyClass': df2.loc[Av_Fl[f-1], 'PC_Oversold'], 'EconomyClass': df2.loc[Av_Fl[f-1], 'EC_Oversold']}  

# Initialize count for each class
class_count = {category: 0 for category in class_categories}

# Create a new column 'Assigned Class'
filt_sorted_2['Assigned Class'] = filt_sorted_2.apply(assign_class, axis=1)

In [ ]:
#Passengers who were alloted the same class in flight 1
filt = (filt_sorted_1['Class'] == filt_sorted_1['Assigned Class'])
same_allotment_1 = filt_sorted_1.loc[filt]   

In [828]:
#Passengers who were alloted the same class in flight 2
filt = (filt_sorted_2['Class'] == filt_sorted_2['Assigned Class'])
same_allotment_2 = filt_sorted_2.loc[filt]

In [ ]:
#Passengers who were downgraded 

In [ ]:
#Flight 1

In [829]:
filt1 = (filt_sorted_1['Class'] == 'FirstClass')
filt2 = (filt_sorted_1['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_1['Assigned Class'] == 'EconomyClass')
FC_dn1 = filt_sorted_1.loc[filt1 & (filt2 | filt3| filt4)]

In [830]:
filt2 = (filt_sorted_1['Class'] == 'BusinessClass')
filt3 = (filt_sorted_1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_1['Assigned Class'] == 'EconomyClass')
BC_dn1 = filt_sorted_1.loc[filt2 & (filt3 | filt4)]

In [831]:
filt3 = (filt_sorted_1['Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_1['Assigned Class'] == 'EconomyClass')
PC_dn1 = filt_sorted_1.loc[filt3 & (filt4)]

In [ ]:
#Flight 2

In [832]:
filt1 = (filt_sorted_2['Class'] == 'FirstClass')
filt2 = (filt_sorted_2['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_2['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_2['Assigned Class'] == 'EconomyClass')
FC_dn2 = filt_sorted_2.loc[filt1 & (filt2 | filt3| filt4)]

In [833]:
filt2 = (filt_sorted_2['Class'] == 'BusinessClass')
filt3 = (filt_sorted_2['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_2['Assigned Class'] == 'EconomyClass')
BC_dn2 = filt_sorted_2.loc[filt2 & (filt3 | filt4)]

In [834]:
filt3 = (filt_sorted_2['Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_2['Assigned Class'] == 'EconomyClass')
PC_dn2 = filt_sorted_2.loc[filt3 & (filt4)]

In [ ]:
#Passengers who were upgraded 

In [ ]:
#Flight 1

In [835]:
filt1 = (filt_sorted_1['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_1['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_1['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_1['Class'] == 'EconomyClass')
EC_up1 = filt_sorted_1.loc[filt4 & (filt1 | filt2| filt3)]

In [836]:
filt1 = (filt_sorted_1['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_1['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_1['Class'] == 'PremiumEconomyClass')
PC_up1 = filt_sorted_1.loc[filt3 & (filt1 | filt2)]

In [837]:
filt1 = (filt_sorted_1['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_1['Class'] == 'BusinessClass')
BC_up1 = filt_sorted_1.loc[filt2 & filt1]

In [ ]:
#Flight 2

In [838]:
filt1 = (filt_sorted_2['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_2['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_2['Assigned Class'] == 'PremiumEconomyClass')
filt4 = (filt_sorted_2['Class'] == 'EconomyClass')
EC_up2 = filt_sorted_2.loc[filt4 & (filt1 | filt2| filt3)]

In [839]:
filt1 = (filt_sorted_2['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_2['Assigned Class'] == 'BusinessClass')
filt3 = (filt_sorted_2['Class'] == 'PremiumEconomyClass')
PC_up2 = filt_sorted_2.loc[filt3 & (filt1 | filt2)]

In [840]:
filt1 = (filt_sorted_2['Assigned Class'] == 'FirstClass')
filt2 = (filt_sorted_2['Class'] == 'BusinessClass')
BC_up2 = filt_sorted_2.loc[filt2 & filt1]

In [841]:
print(f"{len(same_allotment_1) + len(same_allotment_2)} passengers were allotted the same class they applied for")
print(f"{len(FC_dn1) + len(BC_dn1) + len(PC_dn1) + len(FC_dn2) + len(BC_dn2) + len(PC_dn2)} passengers were downgraded")
print(f"{len(EC_up1) + len(PC_up1) + len(BC_up1) + len(EC_up2) + len(PC_up2) + len(BC_up2)} passengers were upgraded")

94 passengers were allotted the same class they applied for
65 passengers were downgraded
96 passengers were upgraded
